In [1]:
import tensorflow as tf
from tensorflow import keras
import os
# Chay bang CPU thay vi GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

class buildmodel():
    def conv_block(self,inputs , filters, block_num,conv_num=2):
        x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_1'%block_num)(inputs)
        x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_2'%block_num)(x)
        if conv_num==3:
            x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_3'%block_num)(x)
        x = keras.layers.MaxPooling2D(name = 'max_pooling_%d'%block_num)(x)
        return x

    def vgg16(self,num_outputs=1000):
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = self.conv_block(inputs,64,1)
        x = self.conv_block(x,128,2)
        x = self.conv_block(x,256,3,3)
        x = self.conv_block(x,512,4,3)
        x = self.conv_block(x,512,5,3)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_1')(x)
        x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_2')(x)
        outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'vgg16')
        return model

    def residual_block(self,inputs, filter,block_num,reduce=False):
        res = inputs
        x = keras.layers.Conv2D(filter,(3, 3), strides = 2 if reduce else 1,activation = 'relu', padding = 'same',name = f'conv_{block_num}_1')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_2')(x)
        if reduce == True:
            res = keras.layers.Conv2D(filter,(1, 1), strides = 2, activation = 'relu', padding = 'same',name = f'conv_skip_{block_num-1}')(res)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num-1}')([x,res])
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_3')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_4')(x)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num}')([x,res])
        return res
        
    def resnet18(self,num_outputs=1000):
        # Inputs
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = keras.layers.Conv2D(64,(7, 7),strides = 2, activation = 'relu', padding = 'same',name = 'conv2D')(inputs)
        x = keras.layers.MaxPooling2D(pool_size = (3,3),strides = 2 ,padding = 'same',name = 'max_pool_1')(x)
        # Block 1
        x = self.residual_block(x,64,1,False)
        # Block 2
        x = self.residual_block(x,128,2,True)
        # Block 3
        x = self.residual_block(x,256,3,True)
        # Block 4
        x = self.residual_block(x,512,4,True)
        # Ouputs
        x = keras.layers.GlobalAveragePooling2D(name = 'global_pooling')(x)
        if num_outputs==1:
            outputs = keras.layers.Dense(num_outputs,activation = 'sigmoid',name = 'outputs_sigmoid')(x)
        else:
            outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs_softmax')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'resnet18')
        return model
    def __init__(self,model_name = 'resnet18',num_outputs=1):
        if model_name == 'resnet18':
            self.model = self.resnet18(num_outputs = num_outputs)
        if model_name == 'vgg16':
            self.model = self.vgg16(num_outputs=num_outputs)
        self.opt = keras.optimizers.Adam()
        self.loss_fn = keras.losses.BinaryCrossentropy()
        self.metric = keras.metrics.BinaryAccuracy()
        self.model.compile(optimizer=self.opt, loss= self.loss_fn)
    @tf.function
    def train_step(self, x, y):
        with tf.GradientTape() as tape:
            logits = self.model(x, training=True)
            loss_value = self.loss_fn(y, logits)
        grads = tape.gradient(loss_value, self.model.trainable_weights)
        # self.metric.update_state(y,logits)
        return grads        
    @tf.function
    def update_grads(self, grads):
        self.opt.apply_gradients(zip(grads, self.model.trainable_weights))

2023-12-26 11:11:17.271943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 11:11:17.271967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 11:11:17.273019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-26 11:11:17.278947: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-26 11:11:17.931008: W tensorflow/compiler/tf2

In [2]:
model = buildmodel().model

2023-12-26 11:11:18.646302: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-26 11:11:18.646326: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: AILAB-PC
2023-12-26 11:11:18.646333: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: AILAB-PC
2023-12-26 11:11:18.646383: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2023-12-26 11:11:18.646401: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2023-12-26 11:11:18.646406: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [3]:
model.input_shape

(None, 224, 224, 3)

In [3]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator()
train_gen = img_gen.flow_from_directory('./../../../Sample Model/data',target_size=(224, 224),class_mode="binary",batch_size=32)

Found 24959 images belonging to 2 classes.


In [5]:
# train_gen.batch_size *=10
a = train_gen.next()

In [8]:
len(train_gen)

25

In [11]:
import time

epochs = 1
model_lib = buildmodel()
# model = model_lib.model
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step,(x_batch_train, y_batch_train) in enumerate(train_gen):
        print(step,x_batch_train.shape, y_batch_train.shape)
        print(model_lib.model.input_shape)
        grads = model_lib.train_step(x_batch_train, y_batch_train)
        model_lib.update_grads(grads)
        model_lib.model.evaluate(x_batch_train,y_batch_train)
        # print(model_lib.metric.result())
        # model_lib.model.fit(x_batch_train,y_batch_train)
    print('---------------------------------------------')
    # # Display metrics at the end of each epoch.
    # train_acc = train_acc_metric.result()
    # print("Training acc over epoch: %.4f" % (float(train_acc),))

    # # Reset training metrics at the end of each epoch
    # train_acc_metric.reset_states()

    # print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
0 (32, 224, 224, 3) (32,)
(None, 224, 224, 3)
1/1 [==============================] - 0s 378ms/step - loss: 33.3523
1 (32, 224, 224, 3) (32,)
(None, 224, 224, 3)
1/1 [==============================] - 0s 246ms/step - loss: 37.6095
2 (32, 224, 224, 3) (32,)
(None, 224, 224, 3)
1/1 [==============================] - 0s 282ms/step - loss: 36.1967
3 (32, 224, 224, 3) (32,)
(None, 224, 224, 3)
1/1 [==============================] - 0s 252ms/step - loss: 44.6596
4 (32, 224, 224, 3) (32,)
(None, 224, 224, 3)
1/1 [==============================] - 0s 245ms/step - loss: 54.7658
5 (32, 224, 224, 3) (32,)
(None, 224, 224, 3)
1/1 [==============================] - 0s 253ms/step - loss: 39.0838
6 (32, 224, 224, 3) (32,)
(None, 224, 224, 3)


KeyboardInterrupt: 

In [10]:
model_lib.model.fit(train_gen,epochs=2)

Epoch 1/2
422/780 [===============>..............] - ETA: 7:24 - loss: 0.7004

/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


780/780 [==============================] - 980s 1s/step - loss: 0.6886
Epoch 2/2
780/780 [==============================] - 980s 1s/step - loss: 0.6570


In [16]:
len(a[0][0])

224

In [19]:
model_lib.model.evaluate(a[0],a[1])

1/1 [==============================] - 0s 407ms/step - loss: 0.5929


0.5928722620010376

# Bản cũ hơn

In [12]:
import tensorflow as tf
from tensorflow import keras
import os
# Chay bang CPU thay vi GPU
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

class buildmodel():
    def conv_block(self,inputs , filters, block_num,conv_num=2):
        x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_1'%block_num)(inputs)
        x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_2'%block_num)(x)
        if conv_num==3:
            x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_3'%block_num)(x)
        x = keras.layers.MaxPooling2D(name = 'max_pooling_%d'%block_num)(x)
        return x

    def vgg16(self,num_outputs=1000):
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = self.conv_block(inputs,64,1)
        x = self.conv_block(x,128,2)
        x = self.conv_block(x,256,3,3)
        x = self.conv_block(x,512,4,3)
        x = self.conv_block(x,512,5,3)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_1')(x)
        x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_2')(x)
        outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'vgg16')
        return model

    def residual_block(self,inputs, filter,block_num,reduce=False):
        res = inputs
        x = keras.layers.Conv2D(filter,(3, 3), strides = 2 if reduce else 1,activation = 'relu', padding = 'same',name = f'conv_{block_num}_1')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_2')(x)
        if reduce == True:
            res = keras.layers.Conv2D(filter,(1, 1), strides = 2, activation = 'relu', padding = 'same',name = f'conv_skip_{block_num-1}')(res)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num-1}')([x,res])
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_3')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_4')(x)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num}')([x,res])
        return res
        
    def resnet18(self,num_outputs=1000):
        # Inputs
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = keras.layers.Conv2D(64,(7, 7),strides = 2, activation = 'relu', padding = 'same',name = 'conv2D')(inputs)
        x = keras.layers.MaxPooling2D(pool_size = (3,3),strides = 2 ,padding = 'same',name = 'max_pool_1')(x)
        # Block 1
        x = self.residual_block(x,64,1,False)
        # Block 2
        x = self.residual_block(x,128,2,True)
        # Block 3
        x = self.residual_block(x,256,3,True)
        # Block 4
        x = self.residual_block(x,512,4,True)
        # Ouputs
        x = keras.layers.GlobalAveragePooling2D(name = 'global_pooling')(x)
        if num_outputs==1:
            outputs = keras.layers.Dense(num_outputs,activation = 'sigmoid',name = 'outputs_sigmoid')(x)
        else:
            outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs_softmax')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'resnet18')
        return model
    def compiled_model(self,model_name = 'resnet18',num_outputs=1):
        if model_name == 'resnet18':
            model = self.resnet18(num_outputs = num_outputs)
        if model_name == 'vgg16':
            model = self.vgg16(num_outputs=num_outputs)
        model.compile(optimizer=keras.optimizers.Adam(),loss=keras.losses.BinaryCrossentropy())
        return model
    def __init__(self,model_name = 'resnet18',num_outputs=1):
        if model_name == 'resnet18':
            self.model = self.resnet18(num_outputs = num_outputs)
        if model_name == 'vgg16':
            self.model = self.vgg16(num_outputs=num_outputs)
        self.opt = keras.optimizers.Adam()
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits= True)
        # self.metric = keras.metrics.BinaryAccuracy()
        self.model.compile(optimizer=self.opt,loss=self.loss_fn,metrics=keras.metrics.BinaryAccuracy())
    # @tf.function
    # def train_step(self,x, y):
    #     with tf.GradientTape() as tape:
    #         logits = self.model(x, training=True)
    #         loss_value = self.loss_fn(y, logits)
    #     grads = tape.gradient(loss_value, self.model.trainable_weights)
    #     # self.metric.update_state(y,logits)
    #     return grads        
    # @tf.function
    # def update_grads(self, grads):
    #     self.opt.apply_gradients(zip(grads, self.model.trainable_weights))
    @tf.function
    def train_step(self,x, y):
        with tf.GradientTape() as tape:
            logits = self.model(x, training=True)
            loss_value = self.loss_fn(y, logits)
        grads = tape.gradient(loss_value, self.model.trainable_weights)
        self.opt.apply_gradients(zip(grads, self.model.trainable_weights))
        return loss_value

In [13]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator()
train_gen = img_gen.flow_from_directory('./../../../Sample Model/data',target_size=(224, 224),class_mode="binary",batch_size=32)

Found 24959 images belonging to 2 classes.


In [3]:
a = train_gen.next()

In [21]:
import time

epochs = 1
# model = buildmodel()
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_gen):
        print(step, len(x_batch_train), len(y_batch_train))
        # grads = model.train_step(x_batch_train, y_batch_train)
        # model.update_grads(grads)
        # print(model.metric.result())
        print(model.train_step(x_batch_train,y_batch_train))
        # print(model.model.evaluate(a[0],a[1]))

    # # Display metrics at the end of each epoch.
    # train_acc = train_acc_metric.result()
    # print("Training acc over epoch: %.4f" % (float(train_acc),))

    # # Reset training metrics at the end of each epoch
    # train_acc_metric.reset_states()

    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
0 32 32
tf.Tensor(0.7944356, shape=(), dtype=float32)
1 32 32
tf.Tensor(0.68699193, shape=(), dtype=float32)
2 32 32
tf.Tensor(0.844018, shape=(), dtype=float32)
3 32 32
tf.Tensor(0.8224214, shape=(), dtype=float32)
4 32 32
tf.Tensor(0.7051708, shape=(), dtype=float32)
5 32 32
tf.Tensor(0.68255067, shape=(), dtype=float32)
6 32 32
tf.Tensor(0.63509077, shape=(), dtype=float32)
7 32 32
tf.Tensor(0.73378897, shape=(), dtype=float32)
8 32 32
tf.Tensor(0.713483, shape=(), dtype=float32)
9 32 32
tf.Tensor(0.6854629, shape=(), dtype=float32)
10 32 32
tf.Tensor(0.67304385, shape=(), dtype=float32)
11 32 32
tf.Tensor(0.70452446, shape=(), dtype=float32)
12 32 32
tf.Tensor(0.7184558, shape=(), dtype=float32)
13 32 32
tf.Tensor(0.64322495, shape=(), dtype=float32)
14 32 32
tf.Tensor(0.7199877, shape=(), dtype=float32)
15 32 32
tf.Tensor(0.7156796, shape=(), dtype=float32)
16 32 32
tf.Tensor(0.7015571, shape=(), dtype=float32)
17 32 32
tf.Tensor(0.6945214, shape=(), dtype=float3

KeyboardInterrupt: 

In [19]:
model = buildmodel()
model.model.fit(train_gen)

780/780 [==============================] - 49s 58ms/step - loss: 11.2857 - binary_accuracy: 0.5675


In [20]:
model.model.evaluate(train_gen)

780/780 [==============================] - 29s 37ms/step - loss: 0.6746 - binary_accuracy: 0.5523


[0.6746149659156799, 0.5523057579994202]

In [22]:
model.model.evaluate(train_gen)

780/780 [==============================] - 29s 37ms/step - loss: 40.8554 - binary_accuracy: 0.5004


[40.855445861816406, 0.5004206895828247]